In [11]:
import pandas as pd
import plotly.express as px

In [4]:
check_balance_df = pd.read_csv('../csv/check_balance.csv')
paynow_transfer_df = pd.read_csv('../csv/paynow_transfer.csv')
scan_to_pay_df = pd.read_csv('../csv/scan_to_pay.csv')

In [5]:
combined_df = pd.concat([check_balance_df, paynow_transfer_df, scan_to_pay_df], ignore_index=True)

In [6]:
combined_df

,bank_function,date,account_number,see_past_transactions,msgs_to_customer_support,transaction_time,recipient_number,amount,qr_string
0,Check Balance,1/6/23,12345678.0,Past 7 days,"[""I have a question about my recent transactio...",NaN,NaN,NaN,NaN
1,Check Balance,1/6/23,23456789.0,Past 1 month,"[""I'm locked out of my online banking account....",NaN,NaN,NaN,NaN
2,Check Balance,2/6/23,34567890.0,Past 3 months,"[""I suspect fraud on my account. What should I...",NaN,NaN,NaN,NaN
3,Check Balance,2/6/23,45678901.0,Past 7 days,"[""I received a suspicious email. Is it from th...",NaN,NaN,NaN,NaN
4,Check Balance,2/6/23,56789012.0,Past 1 month,"[""I'm experiencing issues with online banking....",NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
143,Scan To Pay,1/8/23,45678901.0,NaN,"[""I have a question about a recent transaction...",10:45:55,98888890.0,1000.75,QkVHSU46VkFMVUVTVC1QVUJMSUMgUE5HOjpBbW91bnQ6MT...
144,Scan To Pay,1/8/23,56789012.0,NaN,"[""Can you provide the wire transfer fee detail...",11:30:55,96666667.0,800.40,QkVHSU46VkFMVUVTVC1QVUJMSUMgUE5HOjpBbW91bnQ6MT...
145,Scan To Pay,1/8/23,67890123.0,NaN,"[""How often should I change my online banking ...",15:20:10,98888891.0,450.55,QkVHSU46VkFMVUVTVC1QVUJMSUMgUE5HOjpBbW91bnQ6MT...
146,Scan To Pay,1/8/23,12345678.0,NaN,"[""What's the maximum loan amount for small bus...",8:05:45,95555558.0,2200.90,QkVHSU46VkFMVUVTVC1QVUJMSUMgUE5HOjpBbW91bnQ6MT...


In [9]:
bank_function_count = combined_df.groupby(["bank_function"]).size().reset_index(name="bank_function_count")

In [10]:
bank_function_count

,bank_function,bank_function_count
0,Check Balance,48
1,PayNow Transfer,50
2,Scan To Pay,49


In [14]:
fig = px.pie(
    bank_function_count,
    values="bank_function_count",
    names="bank_function",
    title="Bank Function Distribution",
    hole=0.6,
    color_discrete_sequence=["#B8D5E5", "#92BFD8", "#63A3C7"]
)
fig.show()

In [31]:
from transformers import pipeline
import pandas as pd
import re
import time

start = time.time()

sentiment_pipeline = pipeline("sentiment-analysis") #create sentiment analysis model
df = pd.read_csv('../csv/overview.csv') #read overview df
unrecognised_msgs = df["unrecognised_msgs"].tolist() #get unrecognised_msgs column

msgs = []
pattern = r'["\'\[\]]'

for lst in unrecognised_msgs:
    lst = lst.split(", ") # transform string representation of array to array ie "[['msg1', 'msg2'], ['msg3', 'msg4']]" to [['msg1', 'msg2'], ['msg3', 'msg4']]
    for msg in lst:
            msg = re.sub(pattern, '', msg) # remove ' " [ ] from msg
            msg = msg.strip() # remove spaces from the start and end of msg
            if msg != "": # msgs_to_customer_support column is []
                msgs.append(msg)

sentiments = sentiment_pipeline(msgs) # apply sentiment analysis

# turn msgs and sentiment into a csv
msgs_sentiments = {"unrecognised_msgs": msgs, "sentiment_label_score": sentiments}
msgs_sentiments_df = pd.DataFrame(msgs_sentiments)
msgs_sentiments_df.to_csv("../csv/sentiments.csv", index=False)

end = time.time()
total_seconds = end - start
minutes = total_seconds // 60
seconds = total_seconds % 60
print(f"{minutes:02}:{seconds:02}")


No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
All PyTorch model weights were used when initializing TFDistilBertForSequenceClassification.

All the weights of TFDistilBertForSequenceClassification were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertForSequenceClassification for predictions without further training.


0.0:15.521202087402344


In [34]:
import json
sentiment_df = pd.read_csv("../csv/sentiments.csv")
sentiment_df["sentiment_label_score"] = sentiment_df["sentiment_label_score"].apply(lambda x: x.replace("'", "\""))
sentiment_df["sentiment_label"] = sentiment_df["sentiment_label_score"].apply(lambda x: json.loads(x)["label"])
sentiment_df.head(5)

,unrecognised_msgs,sentiment_label_score,sentiment_label
0,I have a question about my recent transactions.,"{""label"": ""NEGATIVE"", ""score"": 0.8892908096313...",NEGATIVE
1,How can I set up e-statements?,"{""label"": ""NEGATIVE"", ""score"": 0.999462902545929}",NEGATIVE
2,Can you help me with my credit card rewards?,"{""label"": ""NEGATIVE"", ""score"": 0.8585994243621...",NEGATIVE
3,Im locked out of my online banking account.,"{""label"": ""NEGATIVE"", ""score"": 0.9991946816444...",NEGATIVE
4,Whats the process for applying for a credit card?,"{""label"": ""NEGATIVE"", ""score"": 0.9964313507080...",NEGATIVE


In [36]:
sentiment_distribution = sentiment_df.groupby(["sentiment_label"]).size().reset_index(name="sentiment_distribution")
sentiment_distribution

,sentiment_label,sentiment_distribution
0,NEGATIVE,127
1,POSITIVE,5


In [38]:
fig = px.pie(
    sentiment_distribution,
    values="sentiment_distribution",
    names="sentiment_label",
    title="Sentiment Distribution",
    hole=0.6,
    color_discrete_sequence=["#B8D5E5", "#92BFD8"]
)
fig.show()

In [ ]:
check_balance_df = pd.read_csv('../csv/check_balance.csv')
paynow_transfer_df = pd.read_csv('../csv/paynow_transfer.csv')
scan_to_pay_df = pd.read_csv('../csv/scan_to_pay.csv')

In [ ]:
def get_fig(df, metric_choice):
            df['date'] = pd.to_datetime(df['date'], format="%d/%m/%y")
            sorted_df = df.sort_values(by="date")
            
            if metric_choice == "Users":
                users_df = sorted_df.groupby(["date"])['account_number'].nunique().reset_index(name="user count")
                fig = px.line(users_df, x="date", y="user count", title="Number of Users Over Time", hover_name=["date", "user count"])
            
            elif metric_choice == "Sessions":
                sessions_df = sorted_df.groupby(["date"]).size().reset_index(name="session count")
                fig = px.line(sessions_df, x="date", y="session count", title="Number of Sessions Over Time", hover_name=["date", "session count"])
            
            fig.update_xaxes(rangeslider_visible=True)
            return fig

In [62]:
users_df_arr = []
sessions_df_arr = []

for bf in ["check_balance", "paynow_transfer", "scan_to_pay"]:
    df = pd.read_csv('../csv/' + bf + '.csv')
    df['date'] = pd.to_datetime(df['date'], format="%d/%m/%y")
    sorted_df = df.sort_values(by="date")

    users_df = sorted_df.groupby(["date"])['account_number'].nunique().reset_index(name= bf + "_user_count")
    sessions_df = sorted_df.groupby(["date"]).size().reset_index(name= bf + "_session_count")

    users_df_arr.append(users_df)
    sessions_df_arr.append(sessions_df)

# users_df_final = pd.concat(users_df_arr, axis=1, ignore_index=True)
# sessions_df_final = pd.concat(sessions_df_arr, axis=1, ignore_index=True)

users_df_final = pd.DataFrame({"date": []})
for df in users_df_arr:
    users_df_final = pd.merge(users_df_final, df, on='date', how='outer')

sessions_df_final = pd.DataFrame({"date": []})
for df in sessions_df_arr:
    sessions_df_final = pd.merge(sessions_df_final, df, on='date', how='outer')

In [63]:
users_df_final.head(5)

,date,check_balance_user_count,paynow_transfer_user_count,scan_to_pay_user_count
0,2023-06-01,2,2,2
1,2023-06-02,3,3,3
2,2023-06-03,4,4,4
3,2023-07-01,6,6,6
4,2023-07-02,7,8,6


In [64]:
sessions_df_final.head(5)

,date,check_balance_session_count,paynow_transfer_session_count,scan_to_pay_session_count
0,2023-06-01,2,2,2
1,2023-06-02,3,3,3
2,2023-06-03,4,4,4
3,2023-07-01,6,6,6
4,2023-07-02,8,8,8


In [83]:
check_balance_df = pd.read_csv('../csv/check_balance.csv')
paynow_transfer_df = pd.read_csv('../csv/paynow_transfer.csv')
scan_to_pay_df = pd.read_csv('../csv/scan_to_pay.csv')

metric_choice="Users"

def process_bank_function(df, metric_choice):
    df['date'] = pd.to_datetime(df['date'], format="%d/%m/%y")
    df.sort_values(by="date", inplace=True)

    if metric_choice == "Users":
        return df.groupby(["date"])['account_number'].nunique().reset_index()
    
    elif metric_choice == "Sessions":
        return df.groupby(["date"]).size().reset_index()
    
final_df = pd.DataFrame({"date": []})
for df in [check_balance_df, paynow_transfer_df, scan_to_pay_df]:
    dff = process_bank_function(df, metric_choice)
    final_df = pd.merge(final_df, dff, on='date', how='outer')

overview_user_metrics_fig = px.line(final_df, x='date', y=final_df.columns)
overview_user_metrics_fig.update_layout(yaxis_title='user count')
overview_user_metrics_fig.update_traces(line={'width': 2})

In [ ]:
# generate user metrics chart
def get_fig(df, metric_choice):
    df['date'] = pd.to_datetime(df['date'], format="%d/%m/%y")
    sorted_df = df.sort_values(by="date")
    
    if metric_choice == "Users":
        df = sorted_df.groupby(["date"])['account_number'].nunique().reset_index(name="user count")
        # fig = px.line(users_df, x="date", y="user count", title="Number of Users Over Time")
    
    elif metric_choice == "Sessions":
        df = sorted_df.groupby(["date"]).size().reset_index(name="session count")
        # fig = px.line(sessions_df, x="date", y="session count", title="Number of Sessions Over Time")
    return df
    
fig = px.line(sessions_df, x="date", y="session count", title="Number of {} Over Time".format(metric_choice))
fig.update_xaxes(rangeslider_visible=True)

In [ ]:
def process_bank_function(df, metric_choice, bf):
        df['date'] = pd.to_datetime(df['date'], format="%d/%m/%y")
        df.sort_values(by="date", inplace=True)

        if metric_choice == "Users":
            return df.groupby(["date"])['account_number'].nunique().reset_index(name=bf)
        
        elif metric_choice == "Sessions":
            return df.groupby(["date"]).size().reset_index(name=bf)
    
bf_csv = {"check balance": check_balance_df, "paynow transfer": paynow_transfer_df, "scan to pay": scan_to_pay_df}
bf_colour = {'check balance': '#92C0D8', 'paynow transfer': '#FDCA8C', 'scan to pay': '#9DD4A3'}

final_df = pd.DataFrame({"date": []})
for bf in bf_csv:
    df = bf_csv[bf]
    dff = process_bank_function(df, metric_choice, bf)
    final_df = pd.merge(final_df, dff, on='date', how='outer')

In [11]:
## big query connector
import pandas_gbq
from google.oauth2 import service_account
import pandas as pd

import time
start_time = time.time()

credentials = service_account.Credentials.from_service_account_file('smu-fyp-396613-6aefbba11d63.json')
project_id = 'smu-fyp-396613'

check_balances_df_sql = f"""
SELECT *
FROM smufyp.check_balance_sessions
"""

paynow_transfer_df_sql = f"""
SELECT *
FROM smufyp.paynow_transfer_sessions
"""

scan_to_pay_df_sql = f"""
SELECT *
FROM smufyp.scan_to_pay_sessions
"""

overview_df_sql = f"""
SELECT *
FROM smufyp.unrecognized_messages
"""

check_balances_df = pd.read_gbq(check_balances_df_sql, project_id=project_id, dialect='standard', credentials=credentials)
paynow_transfer_df = pd.read_gbq(paynow_transfer_df_sql, project_id=project_id, dialect='standard', credentials=credentials)
scan_to_pay_df = pd.read_gbq(scan_to_pay_df_sql, project_id=project_id, dialect='standard', credentials=credentials)
overview_df = pd.read_gbq(overview_df_sql, project_id=project_id, dialect='standard', credentials=credentials)

end_time = time.time()

time_taken = end_time - start_time
minutes = int(time_taken // 60)
seconds = int(time_taken % 60)

print(f"time taken {minutes} minutes, {seconds} seconds")


time taken 0 minutes, 12 seconds


In [12]:
check_balances_df

,session_id,bank_function,date,account_number,see_past_transactions,session_rating
0,29dcc22,Check Balance,27/09/23,1000,None,5.0
1,0e39bee,Check Balance,27/09/23,2000,None,5.0
2,00c6614,Check Balance,27/09/23,1,None,NaN
3,ccbfe09,Check Balance,27/09/23,2000,None,3.0
4,9079ba7,Check Balance,27/09/23,1000,None,5.0
5,d0486f7,Check Balance,27/09/23,1000,None,NaN
6,7b2c757,Check Balance,27/09/23,1,None,NaN
7,48073e7,Check Balance,27/09/23,1000,None,2.0
8,e1b5881,Check Balance,27/09/23,1000,yes 7 days,5.0
9,f950486,Check Balance,27/09/23,1000,yes 7 days,2.5


In [13]:
paynow_transfer_df

,session_id,bank_function,date,account_number,recipient_number,amount,session_rating
0,50a7072,Paynow Transfer,28/09/23,+6597988922,None,None,NaN
1,0fcd0df,Paynow Transfer,27/09/23,+6597988922,None,None,NaN
2,53599dc,Paynow Transfer,27/09/23,+6597988922,None,None,NaN
3,b1f5321,Paynow Transfer,28/09/23,+6594551972,94551972,200,NaN
4,50e353b,Paynow Transfer,28/09/23,+6597988922,97988922,200,NaN
5,3d4a177,Paynow Transfer,28/09/23,+6597988922,97988922,100,NaN
6,cad2775,Paynow Transfer,28/09/23,+6597988922,97988922,200,NaN
7,fbcea3e,Paynow Transfer,28/09/23,+6597988922,97988922,5,NaN
8,fd133a6,Paynow Transfer,27/09/23,+6597988922,s9819533h,50,3.0
9,920d00d,Paynow Transfer,27/09/23,+6597988922,84821234,50,4.0


In [17]:
paynow_transfer_df.groupby("date").count()

,session_id,bank_function,account_number,recipient_number,amount,session_rating
date,,,,,,
27/09/23,6,6,6,4,4,4
28/09/23,6,6,6,5,5,0


In [14]:
scan_to_pay_df

,session_id,bank_function,date,account_number,recipient_number,amount,session_rating
0,7bfab07,Scan to Pay,28/09/23,+6597988922,201832698H,None,NaN
1,cad2775,Scan to Pay,28/09/23,+6597988922,201832698H,200,NaN
2,50a7072,Scan to Pay,28/09/23,+6597988922,201832698H,200,NaN
3,3d4a177,Scan to Pay,28/09/23,+6597988922,201832698H,200,NaN
4,3792ce0,Scan to Pay,27/09/23,+6597988922,201832698H,200,NaN
5,ab65826,Scan to Pay,28/09/23,+6597988922,201832698H,10.50,NaN
6,fbcea3e,Scan to Pay,28/09/23,+6597988922,201832698H,10.50,NaN


In [15]:
overview_df

,message
0,No
1,No
2,bo
3,hi
4,hi
5,no
6,no
7,End
8,end
9,end


In [25]:
!pip install pyopenssl --upgrade

  Obtaining dependency information for pyopenssl from https://files.pythonhosted.org/packages/f0/e2/f8b4f1c67933a4907e52228241f4bd52169f3196b70af04403b29c63238a/pyOpenSSL-23.2.0-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.0/59.0 kB 1.8 MB/s eta 0:00:00
DEPRECATION: pyodbc 4.0.0-unsupported has a non-standard version number. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pyodbc or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063
DEPRECATION: textract 1.6.4 has a non-standard dependency specifier extract-msg<=0.29.*. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of textract or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063

In [26]:
from transformers import pipeline
import pandas as pd
import re

sentiment_pipeline = pipeline("sentiment-analysis")
msgs = overview_df["message"].tolist()
sentiments = sentiment_pipeline(msgs)

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
All PyTorch model weights were used when initializing TFDistilBertForSequenceClassification.

All the weights of TFDistilBertForSequenceClassification were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertForSequenceClassification for predictions without further training.


In [27]:
sentiments

[{'label': 'NEGATIVE', 'score': 0.9964459538459778},
 {'label': 'NEGATIVE', 'score': 0.9964459538459778},
 {'label': 'POSITIVE', 'score': 0.9894219040870667},
 {'label': 'POSITIVE', 'score': 0.9983267188072205},
 {'label': 'POSITIVE', 'score': 0.9983267188072205},
 {'label': 'NEGATIVE', 'score': 0.9964459538459778},
 {'label': 'NEGATIVE', 'score': 0.9964459538459778},
 {'label': 'NEGATIVE', 'score': 0.9941535592079163},
 {'label': 'NEGATIVE', 'score': 0.9941535592079163},
 {'label': 'NEGATIVE', 'score': 0.9941535592079163},
 {'label': 'POSITIVE', 'score': 0.9958292841911316},
 {'label': 'POSITIVE', 'score': 0.9958292841911316},
 {'label': 'POSITIVE', 'score': 0.9845153093338013},
 {'label': 'POSITIVE', 'score': 0.9845153093338013},
 {'label': 'NEGATIVE', 'score': 0.9995515942573547},
 {'label': 'POSITIVE', 'score': 0.9845153093338013},
 {'label': 'POSITIVE', 'score': 0.9845153093338013},
 {'label': 'NEGATIVE', 'score': 0.9978629946708679},
 {'label': 'POSITIVE', 'score': 0.999863028526

In [28]:
msgs_sentiments = {"unrecognised_msgs": msgs, "sentiment_label_score": sentiments}
msgs_sentiments_df = pd.DataFrame(msgs_sentiments)
msgs_sentiments_df.to_csv("../csv/sentiments.csv", index=False)

In [31]:
sentiments_df = pd.read_csv("../csv/sentiments.csv")
unrecognised_msgs_df = sentiments_df["unrecognised_msgs"]
unrecognised_msgs_df.to_csv("../csv/unrecognised_msgs.csv", index=False)

In [ ]:
from transformers import pipeline
import pandas as pd
import re
import time

start = time.time()

sentiment_pipeline = pipeline("sentiment-analysis") #create sentiment analysis model
df = pd.read_csv('../csv/overview.csv') #read overview df
unrecognised_msgs = df["unrecognised_msgs"].tolist() #get unrecognised_msgs column

msgs = []
pattern = r'["\'\[\]]'

for lst in unrecognised_msgs:
    lst = lst.split(", ") # transform string representation of array to array ie "[['msg1', 'msg2'], ['msg3', 'msg4']]" to [['msg1', 'msg2'], ['msg3', 'msg4']]
    for msg in lst:
            msg = re.sub(pattern, '', msg) # remove ' " [ ] from msg
            msg = msg.strip() # remove spaces from the start and end of msg
            if msg != "": # msgs_to_customer_support column is []
                msgs.append(msg)

sentiments = sentiment_pipeline(msgs) # apply sentiment analysis

# turn msgs and sentiment into a csv
msgs_sentiments = {"unrecognised_msgs": msgs, "sentiment_label_score": sentiments}
msgs_sentiments_df = pd.DataFrame(msgs_sentiments)
msgs_sentiments_df.to_csv("../csv/sentiments.csv", index=False)

end = time.time()
total_seconds = end - start
minutes = total_seconds // 60
seconds = total_seconds % 60
print(f"{minutes:02}:{seconds:02}")


In [33]:
!pip install --upgrade nltk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 16.9 MB/s eta 0:00:0000:0100:01
DEPRECATION: pyodbc 4.0.0-unsupported has a non-standard version number. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pyodbc or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063
DEPRECATION: textract 1.6.4 has a non-standard dependency specifier extract-msg<=0.29.*. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of textract or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063
  Attempting uninstall: nltk
    Found existing installation: nltk 3.7
    Uninstalling nltk-3.7:
      Successfully uninstalled nltk-3.7


In [ ]:
for df in [paynow_transfer_df, scan_to_pay_df]:
    df['date'] = df['date'].apply(lambda x: x.replace("T", " "))
    df['account_number'] = df['account_number'].apply(lambda x: x[3:])
    df.rename(columns={'session_id': 'Session ID', 'bank_function': 'Bank Function', 'date': 'Date', 'account_number': 'Sender Mobile Number', 'recipient_number': 'Recipient Mobile Number/ NRIC', 'amount': 'Amount', 'session_rating': 'Session Rating'})

for df in [check_balance_df, paynow_transfer_df, scan_to_pay_df]:
    df.fillna("NA", inplace=True)

sentiment_df.rename(columns={'unrecognised_msgs': 'Unrecognized Messages', 'sentiment_label': 'Sentiment Label'}, inplace=True)
check_balance_df.rename(columns={'session_id': 'Session ID', 'bank_function': 'Bank Function', 'date': 'Date', 'account_number': 'Balance', 'see_past_transacitons': '', 'session_rating': 'Session Rating'})



In [6]:
from datetime import datetime

# Your ISO 8601 date and time string
iso_string = "2023-01-28T00:09:00"

# Convert to a datetime object
iso_datetime = datetime.fromisoformat(iso_string)

# Format it as a human-readable string
readable_format = iso_datetime.strftime("%Y-%m-%d %H:%M:%S")

print(readable_format)

2023-01-28 00:09:00


In [8]:
"2023-01-28T00:09:00".replace("T", " ")

'2023-01-28 00:09:00'

In [ ]:
from dash import Dash, dash_table
import pandas as pd
from collections import OrderedDict

data = OrderedDict(
    [
        ("Date", ["2015-01-01", "2015-10-24", "2016-05-10", "2017-01-10", "2018-05-10", "2018-08-15"]),
        ("Region", ["Montreal", "Toronto", "New York City", "Miami", "San Francisco", "London"]),
        ("Temperature", [1, -20, 3.512, 4, 10423, -441.2]),
        ("Humidity", [10, 20, 30, 40, 50, 60]),
        ("Pressure", [2, 10924, 3912, -10, 3591.2, 15]),
    ]
)

df = pd.DataFrame(data)

app = Dash(__name__)

app.layout = dash_table.DataTable(
    data=df.to_dict('records'),
    columns=[
        {"name": i, "id": i} for i in df.columns
    ],
    style_data_conditional=[
        {
            'if': {
                'filter_query': '{{Temperature}} = {}'.format(df['Temperature'].min()),
            },
            'backgroundColor': '#FF4136',
            'color': 'white'
        },
    ]
)

if __name__ == '__main__':
    app.run(debug=True)


In [ ]:
import dash
from dash import dash_table
import pandas as pd

# Sample data
data = {
    'Sentiment Label': ['POSITIVE', 'NEGATIVE', 'POSITIVE', 'NEUTRAL'],
    'Text': ['Good', 'Bad', 'Great', 'Okay']
}
df = pd.DataFrame(data)

app = dash.Dash(__name__)

app.layout = dash_table.DataTable(
    id='table',
    columns=[
        {"name": "Sentiment Label", "id": "Sentiment Label"},
        {"name": "Text", "id": "Text"},
    ],
    data=df.to_dict('records'),
    style_data_conditional=[
        {
            'if': {
                'filter_query': '{Sentiment Label} eq "POSITIVE"',
            },
            'backgroundColor': '#B4DBB4',
        },
    ],
    style_table={'height': '300px', 'overflowY': 'auto'},
    style_as_list_view=True
)

if __name__ == '__main__':
    app.run_server(debug=True)
